***ECE685D***

**Dima Tsvetkov**

**NetID: dt169**

Agreement 1) This assignment represents my own work. I did not work on this assignment with
others. All coding was done by myself.

Agreement 2) I understand that if I struggle with this assignment that I will reevaluate whether
this is the correct class for me to take. I understand that the homework only gets harder.

***Problem 1: Linear regression on a simple dataset(30 pts)***
**Part (1)**

In [2]:
import numpy as np
import pandas as pd

In [3]:
XY_raw = pd.read_csv('Concrete_Data_Yeh.csv')
XY_raw

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


Processing the data.

In [4]:
X = XY_raw.iloc[:, :-1]
y = XY_raw.iloc[:, -1]

First, let's find the derivatives of MSE loss function with respect to weights (including the bias $\beta_0$).
$$MSE=\frac{1}{N}\sum_{i=1}^{N}(Y_i - X_i\beta - \beta_0)^2,$$
or
$$MSE=\frac{1}{N}||(Y - X\beta - \beta_0)||^2.$$
To further simplify the expression let's put $\beta_0$ into the vector $\beta$. For that, we can add array of 1's into the X as an extra feature. in that case our loss is:
$$MSE=\frac{1}{N}||(Y - X\beta)||^2.$$
Thus, the derivative with respect to weights $\beta$:
$$\frac{\partial MSE}{\partial \beta}=\frac{1}{N}X^T(Y - X\beta).$$
Setting the derivative of MSE to zero (in order to get the minimum) and rearranging it, we can get:
$$\frac{1}{N}X^T(Y - X\beta)=0,$$
$$X^TY=X^TX\beta,$$
and the expression for $\beta$ is
$$\beta=(X^TX)^{-1}X^TY.$$
Let's implement these calculations.

In [14]:
# implementing the bios term into the X array
X_new = X.copy()
X_new['bias'] = np.ones(len(y))

def LR_multi_get_coefficients(X, y):
    """
    getting coefficients for LR using the exact solution of the minimization problem:
    $$\beta=(X^TX)^{-1}X^TY.$$
    """
    XT_X = np.dot(X.T, X)
    XT_X_inv = np.linalg.inv(XT_X)
    XT_X_inv_XT = np.dot(XT_X_inv, X.T)
    beta = np.dot(XT_X_inv_XT, y)
    return beta

def y_predicted(X, beta):
    # calculation of the predicted values using LR
    return np.dot(X, beta.T)

Now, let's use our solution to predict the values $Y$ and calculate MSE.

In [30]:
def MSE(y, y_pred):
    # MSE Loss function for LR
    dif = y - y_pred
    loss = np.dot(dif.T, dif) / len(y)
    return loss

beta = LR_multi_get_coefficients(X_new, y)
y_pred = y_predicted(X_new, beta)

print('First 5 predicted values vs the real values:')
print(f'{list(y[:5])}\n{y_pred[:5]}')
print('*'*80)
print(f'Loss function value: {MSE(y, y_pred)}')
print('*'*80)
print(f'Coefficients \u03B2:\n{beta[:8]}\nBias: {beta[8]}')

First 5 predicted values vs the real values:
[79.99, 61.89, 40.27, 41.05, 44.3]
[53.46346329 53.73475651 56.81258504 67.66368153 60.91205585]
********************************************************************************
Loss function value: 107.19723607486016
********************************************************************************
Coefficients β:
[ 0.11980433  0.10386581  0.08793432 -0.14991842  0.2922246   0.01808621
  0.02019035  0.11422207]
Bias: -23.33121358503561


We can do the same with calculation the actual derivatives of MSE function:
$$\frac{\partial MSE}{\partial \beta}=-\frac{2}{N}||Y - X\beta||^T X$$
and using the gradient descent with appropriate learning rates.

In [41]:
def gradient_beta(X, y, beta):
    # d MSE/d beta
    dif = y - y_predicted(X, beta)
    grad = -2 * np.dot(dif.T, X) / len(y)
    return grad

beta2 = np.zeros(np.shape(X_new)[1])
# have to set different learning rates for beta and bias terms, otherwise I need too many steps
learning_rate = 0.0000001
learning_rate_bias = 0.05
for i in range(20000):
    gradient = gradient_beta(X_new, y, beta2)
    beta2[:8] -= learning_rate * gradient[:8]
    beta2[8] -= learning_rate_bias * gradient[8]

We can see, that the second approach is getting close to the exact solution. To get rid of the small difference in the loss function values we, probably, need to preprocess the data (at least normalize all the features to the same diapason) and play with the learning rates.
At least I am happy since it also works.

In [42]:
y_pred2 = y_predicted(X_new, beta2)
print(f'Loss function value (exact solution):\n{MSE(y, y_pred)}')
print(f'Loss function value (gradients):\n{MSE(y, y_pred2)}')

Loss function value (exact solution):
107.19723607486016
Loss function value (gradients):
108.1293920372973


**Part (2)**

Let's split the data randomly into the training and test parts.

In [47]:
X_new_shuffles = X_new.copy().sample(frac=1, random_state=1)
y_shuffles = y.copy().sample(frac=1, random_state=1)
index_75 = int(len(y)*0.75)
X_train, X_test = X_new_shuffles[:index_75], X_new_shuffles[index_75:]
y_train, y_test = y_shuffles[:index_75], y_shuffles[index_75:]

Let's randomly choose 6 and 7 independent variables for i={7,8} (We will keep the bias for all 3 models with i={7,8,9}).

In [132]:
beta_m1 = np.arange(9)
rng=np.random.RandomState(4)
beta_m2_no_bias = np.sort(rng.choice(8, 7, replace=False))
beta_m2 = np.concatenate((beta_m2_no_bias, [8]))
beta_m3_no_bias = np.sort(rng.choice(8, 6, replace=False))
beta_m3 = np.concatenate((beta_m3_no_bias, [8]))
print(beta_m1, beta_m2, beta_m3)

[0 1 2 3 4 5 6 7 8] [0 1 3 4 5 6 7 8] [0 3 4 5 6 7 8]


Now let's find coefficients for all 3 models and calculate losses on the test data sets.

In [133]:
beta1 = LR_multi_get_coefficients(X_train.iloc[:, beta_m1], y_train)
y_pred1 = y_predicted(X_test.iloc[:, beta_m1], beta1)
print('9 parameters MSE Loss: ', MSE(y_test, y_pred1))
print('*'*80)
beta2 = LR_multi_get_coefficients(X_train.iloc[:, beta_m2], y_train)
y_pred2 = y_predicted(X_test.iloc[:, beta_m2], beta2)
print('8 parameters MSE Loss: ', MSE(y_test, y_pred2))
print('*'*80)
beta3 = LR_multi_get_coefficients(X_train.iloc[:, beta_m3], y_train)
y_pred3 = y_predicted(X_test.iloc[:, beta_m3], beta3)
print('7 parameters MSE Loss: ', MSE(y_test, y_pred3))


9 parameters MSE Loss:  111.10161646985645
********************************************************************************
8 parameters MSE Loss:  114.18549582522323
********************************************************************************
7 parameters MSE Loss:  123.97931131450196


As we can see, with this data it seems like that the best result can be achieved with the higher number of independent parameters. I've tried it for several random choices of the parameters (was changing rng=np.random.RandomState(4)). This result still stays for that data set. 9 is better than 8, 8 is better than 7.

I believe, models with multiple linear regression that include more independent variables do not always perform better because of the overfitting. However, It seems like the MSE R^2 Error always will be smaller with more independent variables (found it in several places on the internet. For example here https://stats.stackexchange.com/questions/306267/is-mse-decreasing-with-increasing-number-of-explanatory-variables and https://statisticsbyjim.com/regression/interpret-adjusted-r-squared-predicted-r-squared-regression/). But it does not mean it's better to have more variables even tho MSE is decreasing. It's necessary to look at other metrics as well.